## Learning-to-rank: LambdaMART gradient boosting implementation

https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_svmlight_file
from tqdm.notebook import tqdm
from scipy.special import expit
import gc

In [2]:
X, y, query_ids = load_svmlight_file('data/l2r/train.txt', query_id=True)
X = X.toarray()
gc.collect()
remove_zero_mask = ~np.all(X == 0, axis=0)
X = X[:,remove_zero_mask].astype('float32')
gc.collect()

0

In [12]:
def dcg(y):
    return np.sum((2**y - 1)/ np.log2(np.arange(2, len(y) + 2)))

def dcg_k(y, k=5):
    k = min(k, len(y))
    y = y[:k]
    return dcg(y) 

def idcg_k(y, k=5):
    y = np.sort(y)[::-1]
    return dcg_k(y, k)

def idcg(y):
    y = np.sort(y)[::-1]
    return dcg(y)

                 
def ndcg_k(y, k=5):
    k = min(k, len(y))
    idcg_val = idcg_k(y, k)
    if idcg_val == 0:
        return 0
    else:
        return dcg_k(y, k) / idcg_val

def ndcg_dataset(scores, y, query_ids, k=5):
    qids, qid_counts = np.unique(query_ids, return_counts=True)
    qids = qids[qid_counts > 1]
    ndcg_arr = []
    for qid in qids:
        mask = query_ids == qid
        y_qid = y[mask]
        scores_qid = scores[mask]
        y_pred_qid = y_qid[np.argsort(scores_qid)[::-1]]
        ndcg_arr.append(ndcg_k(y_pred_qid, k))
    
    return np.mean(ndcg_arr)


class LambdaMartBoosting():
    
    def __init__(self, n_estimators=100, 
                 learning_rate=0.1, 
                 max_depth=5, 
                 max_features='auto',
                 splitter='best',
                 min_samples_split=2,
                 min_samples_leaf=1,
                 subsample=1, 
                 verbose=10):
        
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.splitter = splitter
        self.subsample = subsample
        self.verbose = verbose

                              
    def _get_delta_ndcg(self, y_pred, ideal_dcg, i, j, k=5):
        delta_dcg = (2**y_pred[i] - 2**y_pred[j]) * (1/np.log2(i+2) - 1/np.log2(j+2))
        return np.abs(delta_dcg / ideal_dcg)
            
    
    def _get_lambdas(self, y, query_ids, scores):
        sigma = 1 # коэфициент наклона сигмоиды
        lambdas = np.zeros(len(y))
        d2c = np.zeros(len(y)) # 2-е производные для подсчета ньютоновского шага
        all_indexes = np.arange(len(y))
        
        for qid in np.unique(query_ids):
            indexes = all_indexes[query_ids == qid]
            scores_qid = scores[indexes]
            y_qid = y[indexes]
            lambdas_qid = lambdas[indexes]
            d2c_qid = d2c[indexes]
            
            ideal_dcg = idcg(y_qid)
            
            for i in range(len(y_qid)):
                for j in range(len(y_qid)):
                    if y_qid[i] > y_qid[j]:
                        lambda_val = sigma * 1/(1 + np.exp(sigma * (scores_qid[i] - scores_qid[j])))
                        delta_ndcg = self._get_delta_ndcg(y_qid, ideal_dcg, i, j)
                        
                        lambdas_qid[i] += lambda_val * delta_ndcg
                        lambdas_qid[j] -= lambda_val * delta_ndcg
                        
                        d2c_qid[i] += lambda_val * (1 - lambda_val) * delta_ndcg
                        d2c_qid[j] += lambda_val * (1 - lambda_val) * delta_ndcg
            
            lambdas[indexes] = lambdas_qid
            d2c[indexes] = d2c_qid
                           
        return lambdas, d2c
    
    def _update_leafs_with_newton_step(self, tree, X, lambdas, d2c):
        leaf_sample_indexes = tree.apply(X)
        leafs = np.arange(len(tree.children_left))[tree.children_left == -1]
        
        for leaf in leafs:
            leaf_sample_mask = leaf_sample_indexes == leaf
            d2c_sum = d2c[leaf_sample_mask].sum()
            
            mu = 0
            if d2c_sum != 0:
                lambda_sum = lambdas[leaf_sample_mask].sum()
                mu = lambda_sum / d2c_sum # значение ньютоновского шага для листа
            
            tree.value[leaf, 0, 0] = mu
            
    def fit(self, X, y, query_ids, eval_set=None):
        y = np.array(y)
        query_ids = np.array(query_ids)
        unique_qids = np.unique(query_ids)
        self.trees = []
        scores = np.zeros(len(y))
        if eval_set:
            eval_scores = np.zeros(len(eval_set[1]))
        
        for i in tqdm(range(self.n_estimators)):
            gc.collect()
               
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth, 
                splitter=self.splitter, 
                max_features=self.max_features,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf)
            
            
            lambdas, d2c = self._get_lambdas(y, query_ids, scores)
            tree.fit(X, lambdas)
            self._update_leafs_with_newton_step(tree.tree_, X, lambdas, d2c)
            
            self.trees.append(tree)
            scores += self.learning_rate * tree.predict(X)
    
            if eval_set:
                eval_scores += self.learning_rate * tree.predict(eval_set[0])
            
            if i % self.verbose == 0:
                if eval_set:
                    ndcg_train = ndcg_dataset(scores, y, query_ids)
                    ndcg_val = ndcg_dataset(eval_scores, eval_set[1], eval_set[2])
                    print(f'Iter: {i}, train NDCG: {ndcg_train:.5f}, val NDCG: {ndcg_val:.5f}, diff: {(ndcg_train - ndcg_val):.5f}')
                else:
                    print(f'Iter: {i}, train NDCG: {ndcg_dataset(scores, y, query_ids):.5f}')
        
    def predict(self, X):
        scores = np.zeros(len(X))
        for i in range(self.n_estimators):
            tree = self.trees[i]
            scores += self.learning_rate * tree.predict(X)
        
        return scores
        

In [ ]:
train_size = int(X.shape[0] * 0.9)
lrb = LambdaMartBoosting(n_estimators=2000, 
                         learning_rate=0.1, 
                         max_depth=9, verbose=1)
eval_set = (X[train_size:], y[train_size:], query_ids[train_size:])
lrb.fit(X[:train_size], y[:train_size], query_ids[:train_size], eval_set=eval_set)

In [11]:
lrb = LambdaMartBoosting(n_estimators=1000, 
                         learning_rate=0.05, 
                         max_depth=9)
lrb.fit(X, y, query_ids)

In [9]:
X_sparse_sub, y_sub, query_ids_sub = load_svmlight_file('data/l2r/test.txt', query_id=True)
X_sub = X_sparse_sub.toarray()[:,remove_zero_mask]
gc.collect()
y_sub = lrb.predict(X_sub)

In [10]:
with open('submit40.csv', 'w') as f:
    f.write('QueryId,DocumentId')
    f.write('\n')
    doc_ids = np.arange(1, len(y_sub) + 1)
    for qid in np.unique(query_ids_sub):
        y_sub_qid = y_sub[query_ids_sub == qid]
        doc_ids_qid = doc_ids[query_ids_sub == qid]
        indexes_qid = np.argsort(y_sub_qid)[::-1]
        doc_ids_qid_ordered = doc_ids_qid.take(indexes_qid)
        for doc_id in doc_ids_qid_ordered:
            f.write(f'{qid},{doc_id}')
            f.write('\n')